In [2]:
#code created by Carl van Beek
#last updated on 16-10-2023
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score

In [4]:
#load data
#Data is in long format with columns for EEG spectral power values (baseline-corrected or absolute), Order, Subject, Trial number, and Condition.

Alpha_norm_table = pd.read_csv("C:/Users/.../Table_alpha_baselinecorrectedvalues.csv")
Beta_norm_table = pd.read_csv("C:/Users/.../Table_beta_baselinecorrectedvalues.csv")
Theta_norm_table = pd.read_csv("C:/Users/.../Table_theta_baselinecorrectedvalues.csv")
Engagement_norm_table = pd.read_csv("C:/Users/.../Table_engagement_baselinecorrectedvalues.csv")

Alpha_absolute_table = pd.read_csv("C:/Users/.../Table_alpha_absolutevalues.csv")
Beta_absolute_table = pd.read_csv("C:/Users/.../Table_beta_absolutevalues.csv")
Theta_absolute_table = pd.read_csv("C:/Users/.../Table_theta_absolutevalues.csv")
Engagement_absolute_table = pd.read_csv("C:/Users/.../Table_engagement_absolutevalues.csv")

In [5]:
#create tables containing only features
#new_duty_aggr_table = duty_aggr_table.drop(columns = ['Subject','Trial', 'Condition'])

new_Alpha_norm_table = Alpha_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Beta_norm_table = Beta_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Theta_norm_table = Theta_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Engagement_norm_table = Engagement_norm_table.drop(columns = ['Order','Subject','Trial', 'Condition'])

new_Alpha_absolute_table = Alpha_absolute_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Beta_absolute_table = Beta_absolute_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Theta_absolute_table = Theta_absolute_table.drop(columns = ['Order','Subject','Trial', 'Condition'])
new_Engagement_absolute_table = Engagement_absolute_table.drop(columns = ['Order','Subject','Trial', 'Condition'])

#concatenate all features into a single feature table, making a distinction between a table with and without stick data
full_EEG_norm_table = pd.concat([new_Alpha_norm_table,new_Beta_norm_table,new_Theta_norm_table,new_Engagement_norm_table], axis=1)
#full_norm_table = pd.concat([full_EEG_norm_table,new_duty_aggr_table], axis = 1 )

full_EEG_absolute_table = pd.concat([new_Alpha_absolute_table,new_Beta_absolute_table,new_Theta_absolute_table,new_Engagement_absolute_table], axis=1)
#full_absolute_table = pd.concat([full_EEG_absolute_table,new_duty_aggr_table], axis = 1 )

#get features
y = Alpha_norm_table['Condition'] # prediction labels, they are the same in all tables, alpha_norm is just randomly used

function_test_table = pd.concat([full_EEG_absolute_table,y], axis = 1)
print(function_test_table)

    Alpha_FP1  Alpha_FP2  Alpha_AF3  Alpha_AF4  Alpha_F7  Alpha_F3  Alpha_Fz  \
0    1.013721   1.327482   0.450079   0.544898  0.374854  0.556412  0.553690   
1    0.977242   1.256911   0.405964   0.468593  0.357436  0.487161  0.485874   
2    0.956857   1.230577   0.440352   0.491417  0.377880  0.543356  0.538828   
3    1.044784   1.321664   0.536318   0.602484  0.411094  0.592916  0.639552   
4    0.985492   1.257248   0.533000   0.574808  0.401093  0.656708  0.668012   
..        ...        ...        ...        ...       ...       ...       ...   
67   1.747234   4.015614   0.356219   0.854112  0.442396  0.100220  0.122624   
68   2.072914   4.693699   0.315417   0.930107  0.448993  0.083244  0.102538   
69   1.394316   2.727770   0.328664   0.808801  0.394015  0.089394  0.107687   
70   1.375063   2.713285   0.357637   1.424793  0.439692  0.089038  0.106762   
71   1.409065   2.782531   0.377305   1.343245  0.508259  0.096477  0.111789   

    Alpha_F4  Alpha_F8  Alpha_FC5  ... 

In [6]:
#Dataframe should have columns: 'Condition' and Alpha, Beta, Theta and Engagement values
#for all channels. In the case of 12 x 6 trials and 32 channels, this would be a 72 rows x 129 columns dataframe
#Order, Subject and Trial are not used in this function, but could be used in other applications
def workload_predictor(df):
    X_EEG = df.drop(columns = ['Condition'])
    #Make sure there is a column called 'Condition'. Could be replaced by an integer if there is a standard for that.
    y = df['Condition']
    #first create the RFE algorithm
    SVM = SVC(kernel = 'linear',C = 1,gamma = 0.1) #Choose a classifier to perform RFE
    SVM_RFE = RFE(SVM,n_features_to_select = 10) #Perform RFE
    SVM_RFE.fit(X_EEG,y) 

    #View which features got selected
    selected_EEG = X_EEG.columns[SVM_RFE.support_]
    print("Selected features: ", selected_EEG)


    #Transform the original X-data to only retain the selected features
    X_RFE_EEG = SVM_RFE.transform(X_EEG)

    #global value to keep track of the accuracy of each iteration
    all_accuracies_EEG = []
    all_f1_EEG = []
    all_precision_EEG = []
    all_recall_EEG = []

    #perform the training and testing 10 times, to rule out chance due to the small dataset
    for i in range(1,11):
        #split into 70-30 training and testing for RFE
        X_RFE_train_EEG, X_RFE_test_EEG, y_train, y_test = train_test_split(X_RFE_EEG, y, test_size = 0.3)

        #gridsearch for all models of the first level in the stacking classifier
        rf_params = {'n_estimators': [10, 25, 50, 100,200], 'max_depth': [None, 5, 10, 15]}
        RF_EEG = RandomForestClassifier() #Check literature
        rf_grid_EEG = GridSearchCV(RF_EEG, rf_params, cv=5, n_jobs=-1)
        rf_grid_EEG.fit(X_RFE_train_EEG,y_train)

        svm_params = {'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10, 100]}
        svm_EEG = SVC()
        svm_grid_EEG = GridSearchCV(svm_EEG, svm_params, cv=5, n_jobs=-1)
        svm_grid_EEG.fit(X_RFE_train_EEG,y_train)

        lr_params = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['none','l2']}
        lr_EEG = LogisticRegression()
        lr_grid_EEG = GridSearchCV(lr_EEG, lr_params, cv=5, n_jobs=-1)
        lr_grid_EEG.fit(X_RFE_train_EEG,y_train)

        #variable to input as the first level in the stacking classifier
        estimators_EEG = [('rf', rf_grid_EEG),('svc', svm_grid_EEG),('lr', lr_grid_EEG)]

        #gridsearch for the second level of the stacking model
        second_level_params_EEG = {'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10, 100]}
        second_level_model_EEG = SVC()
        second_level_grid_EEG = GridSearchCV(second_level_model_EEG, second_level_params_EEG, cv=5, n_jobs=-1)
        second_level_grid_EEG.fit(X_RFE_train_EEG,y_train)

        #Create the stacking classifier
        sclf_EEG = StackingClassifier(estimators=estimators_EEG, final_estimator=second_level_grid_EEG)
        sclf_EEG.fit(X_RFE_train_EEG, y_train)

        prediction = sclf_EEG.predict(X_RFE_test_EEG)
        #get model perfromance
        accuracy_EEG = accuracy_score(y_test, prediction)
        f1_EEG = f1_score(y_test, prediction)
        precision_EEG = precision_score(y_test, prediction)
        recall_EEG = recall_score(y_test, prediction)
        #add the performance metrics to their global list to obtain averages later
        all_accuracies_EEG.append(accuracy_EEG)
        all_f1_EEG.append(f1_EEG)
        all_precision_EEG.append(precision_EEG)
        all_recall_EEG.append(recall_EEG)
        #print out the metrics for the current iteration
        print("Accuracy test number ", i, ": ", accuracy_EEG)
        print("F1 score test number ", i, ": ", f1_EEG)
        print("Precision test number ", i, ": ", precision_EEG)
        print("Recall test number ", i, ": ", recall_EEG)
        #obtain the chosen hyperparameters
        print("Chosen parameters for the random forest model in the first level of the stacking classifier for iteration number ",i,": ",rf_grid_EEG.best_params_)
        print("Chosen parameters for the SVM model in the first level of the stacking classifier for iteration number ",i,": ",svm_grid_EEG.best_params_)
        print("Chosen parameters for the Logistic regression model in the first level of the stacking classifier for iteration number ",i,": ",lr_grid_EEG.best_params_)
        print("Chosen parameters for the SVM model in the second level of the stacking classifier for iteration number ",i,": ",second_level_grid_EEG.best_params_)

    #print out the average model evaluation scores of all iterations
    print("Average accuracy score", np.mean(all_accuracies_EEG))
    print("Average f1 score", np.mean(all_f1_EEG))
    print("Average precision", np.mean(all_precision_EEG))
    print("Average recall", np.mean(all_recall_EEG))

    

In [7]:
workload_predictor(function_test_table)

Selected features:  Index(['Alpha_P8', 'Beta_FP2', 'Beta_F8', 'Beta_P8', 'Theta_FP1', 'Theta_T7',
       'Theta_CP6', 'Theta_Oz', 'Engagement_T8', 'Engagement_P8'],
      dtype='object')


C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py

Accuracy test number  1 :  0.6363636363636364
F1 score test number  1 :  0.6363636363636364
Precision test number  1 :  0.6363636363636364
Recall test number  1 :  0.6363636363636364
Chosen parameters for the random forest model in the first level of the stacking classifier for iteration number  1 :  {'max_depth': 10, 'n_estimators': 25}
Chosen parameters for the SVM model in the first level of the stacking classifier for iteration number  1 :  {'C': 10, 'gamma': 1}
Chosen parameters for the Logistic regression model in the first level of the stacking classifier for iteration number  1 :  {'C': 0.01, 'penalty': 'none'}
Chosen parameters for the SVM model in the second level of the stacking classifier for iteration number  1 :  {'C': 10, 'gamma': 1}


C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Accuracy test number  2 :  0.8636363636363636
F1 score test number  2 :  0.8695652173913043
Precision test number  2 :  0.8333333333333334
Recall test number  2 :  0.9090909090909091
Chosen parameters for the random forest model in the first level of the stacking classifier for iteration number  2 :  {'max_depth': None, 'n_estimators': 25}
Chosen parameters for the SVM model in the first level of the stacking classifier for iteration number  2 :  {'C': 10, 'gamma': 1}
Chosen parameters for the Logistic regression model in the first level of the stacking classifier for iteration number  2 :  {'C': 10, 'penalty': 'l2'}
Chosen parameters for the SVM model in the second level of the stacking classifier for iteration number  2 :  {'C': 10, 'gamma': 1}


...

C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the

Accuracy test number  9 :  0.5909090909090909
F1 score test number  9 :  0.6086956521739131
Precision test number  9 :  0.5
Recall test number  9 :  0.7777777777777778
Chosen parameters for the random forest model in the first level of the stacking classifier for iteration number  9 :  {'max_depth': 15, 'n_estimators': 10}
Chosen parameters for the SVM model in the first level of the stacking classifier for iteration number  9 :  {'C': 100, 'gamma': 1}
Chosen parameters for the Logistic regression model in the first level of the stacking classifier for iteration number  9 :  {'C': 0.01, 'penalty': 'none'}
Chosen parameters for the SVM model in the second level of the stacking classifier for iteration number  9 :  {'C': 100, 'gamma': 1}


C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py

Accuracy test number  10 :  0.9090909090909091
F1 score test number  10 :  0.9230769230769231
Precision test number  10 :  0.9230769230769231
Recall test number  10 :  0.9230769230769231
Chosen parameters for the random forest model in the first level of the stacking classifier for iteration number  10 :  {'max_depth': 10, 'n_estimators': 25}
Chosen parameters for the SVM model in the first level of the stacking classifier for iteration number  10 :  {'C': 100, 'gamma': 0.1}
Chosen parameters for the Logistic regression model in the first level of the stacking classifier for iteration number  10 :  {'C': 0.01, 'penalty': 'none'}
Chosen parameters for the SVM model in the second level of the stacking classifier for iteration number  10 :  {'C': 100, 'gamma': 0.1}
Average accuracy score 0.7772727272727272
Average f1 score 0.787228954620259
Average precision 0.795979020979021
Average recall 0.7906737053795877


C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\evyva\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
